In [14]:
import graphgallery as gg
from graphgallery.data import Graph
from graphgallery import functional as gf
from graphgallery.datasets import Planetoid, NPZDataset
import numpy as np,networkx as nx, scipy.sparse as sp
import random, os, pickle,torch,sklearn
import math
from collections import Counter
# data = Planetoid('citeseer', root="~/datasets/datafromgg", verbose=False)
data = NPZDataset('acm', root="~/datasets/datafromgg", verbose=False)
graph = data.graph
splits = data.split_nodes()
random_seed = 5
# print(graph)


#1 SOPT 2 random 3 predictive 4 combination

#Get the lagest subgragh
t = graph.A.toarray()
sg = list(nx.connected_component_subgraphs(nx.from_numpy_matrix(t)))
vid_largest_graph = sg[np.argmax([nx.adjacency_matrix(g).shape[0] for g in sg])].nodes()
adj = t[vid_largest_graph,:]; adj = adj[:, vid_largest_graph]
adj_mat, features, labels = adj, graph.x[vid_largest_graph,:], graph.y[vid_largest_graph]
adj_csr = sp.csr_matrix(adj_mat)
all_x = np.reshape(np.arange(labels.shape[0]), (-1,1))
subgraph = Graph(adj_csr, features, labels)

res_list = [{'vid':random_seed-1, 'test_acc':np.NaN}]
n_sample_acquired = 1
n_sample_budget = 50
tr_mask = np.array([False]*features.shape[0])
tr_mask[random_seed-1] = True
# print(all_x[~tr_mask].flatten())
# print(splits.train_nodes)
result_fp = os.path.join(os.getenv('PWD'), 'al_result_files')
if not (os.path.isdir(result_fp)):
    os.mkdir(result_fp)
result_fp = os.path.join(result_fp, 'GCN_AL-{0}-rs_{1}.p'.format("acm", random_seed))

print("loaded")

# #Test whether it is a uncompleted process
# if os.path.isfile(result_fp):
#     print ('Param. file already exists! Loading from {0}.'.format(result_fp))
#     res_list = pickle.load(open(result_fp, 'rb'))
    
#     n_sample_acquired = len(res_list)
#     ele = 0
#     while ele < n_sample_acquired:
#         tr_mask[res_list[ele]['vid']] = True
#         ele+=1
# print(n_sample_acquired)
# print(len(res_list))
# sum_matrix = np.empty(shape=(0,7))
# out_spikes_counter = torch.Tensor([[ 10.,   0.,   0.,  45.,   0.,   5.,   0.],
#         [  4.,   0.,  72.,   1.,   0.,  10.,   9.],
#         [  0.,   0.,   4., 108.,   0.,   0.,   0.]])
# matrix = out_spikes_counter.numpy()
# sum_matrix = np.concatenate((sum_matrix,out_spikes_counter),axis=0)

# an_array = sklearn.preprocessing.normalize(sum_matrix, norm="l1")
# print(an_array)
# print(x.shape)
# print(torch.softmax(x,dim =1))
#the function to calculate entropy, you should use the probabilities as the parameters
def entropy(score):
    result = np.zeros(score.shape[0])
    ele = 0
    for P in score:
        entropy = 0
        for p in P:
            entropy += (-p)*math.log(p,2)
        result[ele] = entropy
        ele+=1
    return result
def c_entropy(c):
    result=-1;
    if(len(c)>0):
        result=0;
    for x in c:
        result+=(-x)*math.log(x,2)
    return result;

print(len(res_list))     
while n_sample_acquired < n_sample_budget+1:
    #get the accuracy
    trainer = gg.gallery.nodeclas.GCN(seed=None, device="gpu").make_data(subgraph).build()
    his = trainer.fit(all_x[tr_mask].flatten(),verbose=1, dropout=0,epochs=200,ModelCheckpoint=dict(enabled=False))
    score_ac = trainer.evaluate(all_x[~tr_mask].flatten())
    accuracy = score_ac.accuracy
    score = trainer.predict(all_x[~tr_mask].flatten(), transform="softmax")
    print("score:", score.shape)

#     print("entropy:", entropy(score)[0])
#     print("all_x:", all_x)
    
#     #SOPT
#     laplacian = np.diag(np.sum(adj_mat, 1)) - adj_mat
#     predCovCQ = np.zeros((len(laplacian), len(laplacian)))
#     masks = np.reshape(~tr_mask, (-1,1)) & np.reshape(~tr_mask, (1,-1))
#     predCovCQ[masks] = np.linalg.inv(laplacian[~tr_mask][:,~tr_mask]).flatten()
#     acq_scores = np.sum(predCovCQ[~tr_mask][:, ~tr_mask], 1)/np.sqrt(np.diag(predCovCQ)[~tr_mask])
#     to_label = all_x[~tr_mask][np.argmax(acq_scores)]
#     print(to_label)

    #predictive entropy
    p_entropy = entropy(score)
    to_label = all_x[~tr_mask][np.argmax(p_entropy)]
    print(to_label)
    
    
#     #Linear combination
#     laplacian = np.diag(np.sum(adj_mat, 1)) - adj_mat
#     predCovCQ = np.zeros((len(laplacian), len(laplacian)))
#     masks = np.reshape(~tr_mask, (-1,1)) & np.reshape(~tr_mask, (1,-1))
#     predCovCQ[masks] = np.linalg.inv(laplacian[~tr_mask][:,~tr_mask]).flatten()
#     acq_scores = np.sum(predCovCQ[~tr_mask][:, ~tr_mask], 1)/np.sqrt(np.diag(predCovCQ)[~tr_mask])
#     p_entropy = entropy(score)
#     to_label = all_x[~tr_mask][np.argmax(acq_scores+5*p_entropy)]
#      print(to_label)
    #Random
#     to_label = all_x[~tr_mask][random.randint(0,features.shape[0]-1)]
#     print(to_label)
    
    
    #Adding the res_list
    if n_sample_acquired == n_sample_budget:
        res_list[-1]['test_acc'] = accuracy
    else:
        res_list.append({'vid':-100, 'test_acc':np.NaN})
        res_list[-2]['test_acc'], res_list[-1]['vid'] = accuracy, to_label[0] 
        assert np.sum(tr_mask) == n_sample_acquired, 'Num. of sample in tr_mask != n_sample_acquired'
        assert tr_mask[res_list[-1]['vid']]==False, 'Node {0} alrdy acq.'.format(res_list[-1]['vid'])
        tr_mask[res_list[-1]['vid']] = True
    print(n_sample_acquired, " nodes", "accuracy:", accuracy)
    n_sample_acquired += 1
    

import os, pickle

print ('\nSaving result to {0}'.format(result_fp))
pickle.dump(res_list, open(result_fp, 'wb'))
res_list[49]['test_acc']

loaded
1
Training...
200/200 [====================] - Total: 543.38ms - 2ms/step - loss: 0.004 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 55.26ms - 55ms/step - loss: 5.916 - accuracy: 0.361
score: (2005, 3)
[1209]
1  nodes accuracy: 0.36059850454330444
Training...
200/200 [====================] - Total: 536.79ms - 2ms/step - loss: 0.011 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 54.69ms - 54ms/step - loss: 3.111 - accuracy: 0.558
score: (2004, 3)
[1942]
2  nodes accuracy: 0.5583832263946533
Training...
200/200 [====================] - Total: 526.04ms - 2ms/step - loss: 0.057 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.69ms - 53ms/step - loss: 0.690 - accuracy: 0.741
score: (2003, 3)
[975]
3  nodes accuracy: 0.7408886551856995
Training...
200/200 [====================] - Total: 528.31ms - 2ms/step - loss: 0.015 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.47ms - 53ms/step - loss: 1.107 - accuracy:

1/1 [====================] - Total: 52.97ms - 52ms/step - loss: 1.968 - accuracy: 0.376
score: (2001, 3)
[403]
5  nodes accuracy: 0.3758120834827423
Training...
200/200 [====================] - Total: 521.17ms - 2ms/step - loss: 0.022 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.18ms - 53ms/step - loss: 0.897 - accuracy: 0.672
score: (2000, 3)
[44]
6  nodes accuracy: 0.671999990940094
Training...
200/200 [====================] - Total: 530.42ms - 2ms/step - loss: 0.022 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.67ms - 52ms/step - loss: 1.133 - accuracy: 0.633
score: (1999, 3)
[1425]
7  nodes accuracy: 0.6333166360855103
Training...
200/200 [====================] - Total: 535.35ms - 2ms/step - loss: 0.157 - accuracy: 0.875
Testing...
1/1 [====================] - Total: 52.73ms - 52ms/step - loss: 0.617 - accuracy: 0.816
score: (1998, 3)
[1646]
8  nodes accuracy: 0.815815806388855
Training...
200/200 [====================] - Total: 524.36ms

score: (1997, 3)
[981]
9  nodes accuracy: 0.8592889308929443
Training...
200/200 [====================] - Total: 532.29ms - 2ms/step - loss: 0.042 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.81ms - 53ms/step - loss: 0.826 - accuracy: 0.660
score: (1996, 3)
[425]
10  nodes accuracy: 0.6603206396102905
Training...
200/200 [====================] - Total: 522.51ms - 2ms/step - loss: 0.013 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 51.72ms - 51ms/step - loss: 0.389 - accuracy: 0.878
score: (1995, 3)
[515]
11  nodes accuracy: 0.8781954646110535
Training...
200/200 [====================] - Total: 533.61ms - 2ms/step - loss: 0.017 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.76ms - 52ms/step - loss: 0.482 - accuracy: 0.865
score: (1994, 3)
[1419]
12  nodes accuracy: 0.8650952577590942
Training...
200/200 [====================] - Total: 528.81ms - 2ms/step - loss: 0.051 - accuracy: 1.000
Testing...
1/1 [====================] -

1/1 [====================] - Total: 52.98ms - 52ms/step - loss: 0.438 - accuracy: 0.868
score: (1992, 3)
[880]
14  nodes accuracy: 0.8684738874435425
Training...
200/200 [====================] - Total: 888.82ms - 4ms/step - loss: 0.020 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 58.67ms - 58ms/step - loss: 0.471 - accuracy: 0.856
score: (1991, 3)
[630]
15  nodes accuracy: 0.8558513522148132
Training...
200/200 [====================] - Total: 525.70ms - 2ms/step - loss: 0.028 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 54.33ms - 54ms/step - loss: 0.581 - accuracy: 0.847
score: (1990, 3)
[1500]
16  nodes accuracy: 0.847236156463623
Training...
200/200 [====================] - Total: 531.59ms - 2ms/step - loss: 0.060 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.92ms - 53ms/step - loss: 0.423 - accuracy: 0.870
score: (1989, 3)
[1642]
17  nodes accuracy: 0.8702865839004517
Training...
200/200 [====================] - Total: 51

score: (1988, 3)
[519]
18  nodes accuracy: 0.8933601379394531
Training...
200/200 [====================] - Total: 530.88ms - 2ms/step - loss: 0.027 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.33ms - 53ms/step - loss: 0.594 - accuracy: 0.858
score: (1987, 3)
[1727]
19  nodes accuracy: 0.8575742244720459
Training...
200/200 [====================] - Total: 523.73ms - 2ms/step - loss: 0.039 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.70ms - 52ms/step - loss: 0.329 - accuracy: 0.904
score: (1986, 3)
[292]
20  nodes accuracy: 0.904330313205719
Training...
200/200 [====================] - Total: 522.86ms - 2ms/step - loss: 0.041 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.60ms - 52ms/step - loss: 0.415 - accuracy: 0.889
score: (1985, 3)
[870]
21  nodes accuracy: 0.888664960861206
Training...
200/200 [====================] - Total: 530.18ms - 2ms/step - loss: 0.023 - accuracy: 1.000
Testing...
1/1 [====================] - 

...

score: (1970, 3)
[1525]
36  nodes accuracy: 0.8888325095176697
Training...
200/200 [====================] - Total: 557.89ms - 2ms/step - loss: 0.032 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 54.53ms - 54ms/step - loss: 0.413 - accuracy: 0.898
score: (1969, 3)
[1022]
37  nodes accuracy: 0.8984255790710449
Training...
200/200 [====================] - Total: 527.35ms - 2ms/step - loss: 0.036 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.98ms - 52ms/step - loss: 0.427 - accuracy: 0.890
score: (1968, 3)
[632]
38  nodes accuracy: 0.8902438879013062
Training...
200/200 [====================] - Total: 529.34ms - 2ms/step - loss: 0.034 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.98ms - 53ms/step - loss: 0.420 - accuracy: 0.896
score: (1967, 3)
[728]
39  nodes accuracy: 0.8957803845405579
Training...
200/200 [====================] - Total: 544.17ms - 2ms/step - loss: 0.038 - accuracy: 1.000
Testing...
1/1 [====================]

1/1 [====================] - Total: 52.93ms - 52ms/step - loss: 0.522 - accuracy: 0.873
score: (1965, 3)
[472]
41  nodes accuracy: 0.8732824325561523
Training...
200/200 [====================] - Total: 552.18ms - 2ms/step - loss: 0.039 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.89ms - 52ms/step - loss: 0.481 - accuracy: 0.885
score: (1964, 3)
[1349]
42  nodes accuracy: 0.8849287033081055
Training...
200/200 [====================] - Total: 550.68ms - 2ms/step - loss: 0.033 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 52.39ms - 52ms/step - loss: 0.478 - accuracy: 0.877
score: (1963, 3)
[489]
43  nodes accuracy: 0.8767192959785461
Training...
200/200 [====================] - Total: 522.86ms - 2ms/step - loss: 0.070 - accuracy: 0.977
Testing...
1/1 [====================] - Total: 51.79ms - 51ms/step - loss: 0.484 - accuracy: 0.881
score: (1962, 3)
[1197]
44  nodes accuracy: 0.8807339668273926
Training...
200/200 [====================] - Total: 5

score: (1961, 3)
[1082]
45  nodes accuracy: 0.889342188835144
Training...
200/200 [====================] - Total: 920.37ms - 4ms/step - loss: 0.033 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 59.32ms - 59ms/step - loss: 0.541 - accuracy: 0.845
score: (1960, 3)
[638]
46  nodes accuracy: 0.8454081416130066
Training...
200/200 [====================] - Total: 526.84ms - 2ms/step - loss: 0.029 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 54.31ms - 54ms/step - loss: 0.484 - accuracy: 0.881
score: (1959, 3)
[1198]
47  nodes accuracy: 0.880551278591156
Training...
200/200 [====================] - Total: 518.85ms - 2ms/step - loss: 0.049 - accuracy: 1.000
Testing...
1/1 [====================] - Total: 53.26ms - 53ms/step - loss: 0.444 - accuracy: 0.887
score: (1958, 3)
[182]
48  nodes accuracy: 0.8871297240257263
Training...
200/200 [====================] - Total: 518.08ms - 2ms/step - loss: 0.076 - accuracy: 1.000
Testing...
1/1 [====================] -

1/1 [====================] - Total: 53.19ms - 53ms/step - loss: 0.450 - accuracy: 0.882
score: (1956, 3)
[670]
50  nodes accuracy: 0.8819018602371216

Saving result to /home/zlzhu/al_result_files/GCN_AL-acm-rs_5.p


0.8819018602371216

In [15]:
import os, pickle
import numpy as np
result_fp = os.path.join(os.getenv('PWD'), 'al_result_files')
result_fp = os.path.join(result_fp, 'GCN_AL-{0}-rs_{1}.p'.format("acm", 5))
i = 0
accuracy_sum = 0
res_list = pickle.load(open(result_fp, 'rb'))
accuracy = np.zeros(len(res_list))
print(len(res_list))
while i < len(res_list):
    accuracy[i] = res_list[i]['test_acc']
    if (i+1)%5==0:
        print(accuracy[i])
    else: 
        print(accuracy[i], end=" ")
    accuracy_sum += res_list[i]['test_acc']
    i+=1
area = (accuracy_sum*2-res_list[0]['test_acc']-res_list[-1]['test_acc'])/2
curve_rate = area/(49)
print(accuracy)
print(curve_rate)

50
0.36059850454330444 0.5583832263946533 0.7408886551856995 0.6028971076011658 0.3758120834827423
0.671999990940094 0.6333166360855103 0.815815806388855 0.8592889308929443 0.6603206396102905
0.8781954646110535 0.8650952577590942 0.8474661111831665 0.8684738874435425 0.8558513522148132
0.847236156463623 0.8702865839004517 0.8933601379394531 0.8575742244720459 0.904330313205719
0.888664960861206 0.8452621102333069 0.8456883430480957 0.892028272151947 0.8621907830238342
0.8838383555412292 0.8681151866912842 0.8887765407562256 0.8816388249397278 0.89726722240448
0.8946835398674011 0.9072948098182678 0.8859604597091675 0.8651115894317627 0.8990360498428345
0.8888325095176697 0.8984255790710449 0.8902438879013062 0.8957803845405579 0.890640914440155
0.8732824325561523 0.8849287033081055 0.8767192959785461 0.8807339668273926 0.889342188835144
0.8454081416130066 0.880551278591156 0.8871297240257263 0.8717424869537354 0.8819018602371216
[0.3605985  0.55838323 0.74088866 0.60289711 0.37581208 0

In [ ]:
import numpy as np, tensorflow as tf
from scipy import sparse
import torch
import pandas as pd
variance = 1.0 
offset= 8.72369
degree = 3
X =graph.x
# X = tf.reshape(tf.cast(X, tf.int32), [-1])
X2 = X
sparse_P = graph.A.toarray()
sparse_P = sparse_P/np.sum(sparse_P, 1, keepdims=True)
base_K_mat = (variance * X@X.T + offset)**degree
#print(base_K_mat)
t1 = sparse_P@base_K_mat
t2 = sparse_P@t1.T

t2 = t2 / t2.max(axis=0)

k = pd.DataFrame(t2)
u = k.describe()
print("tr_mat.shape()",u)
print(t2)
#tf.gather(t2, X)
#final =tf.gather(tf.gather(t2, X), X2, axis=1)#get the slices

In [96]:
import numpy as np
b=np.arange(50)
print(b)
np.where(b==2)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


(array([2]),)

In [ ]:
import random
a = list(range(10))
random.shuffle(a)
for i in a:
    print(i)

range(0, 50)